## Setup

In [2]:
import networkx as nx

# Ant Colony Optimization

In [5]:
g = nx.read_gml("seidel_rally_graph-12.gml")
g.nodes

NodeView(('Weinstadt', 'Klub Gru', 'Glashütte', 'Das Torberg', 'Mokador 8', 'u.s.w.', 'Pub by Charles', 'Hopfen-Stuben', 'Café Amadeus', 'Acabar', 'Ganz Wien', 'Café 7*stern', 'Kerstin', 'Prater Dome', 'Zum Schlauch', 'Oskar', 'Bastille-Pub', 'Gasthaus Manu', 'Schlupfwinkel', 'Wiener Blut', 'Cafe Nest', 'Derwisch', 'Wirr am Brunnenmarkt', 'Weberknecht', "Wind & Mill's", 'Miranda', 'Top Kino Bar', 'Rendezvous', 'Mimi im Stadtelefant', "Guggi's Beisl", 'Azul Bar', 'Casablanca', 'Slammerbar', 'Red Lion', "'s Stammbeisl", 'Bar Black Lounge', 'Café Frog', 'Gimlet', 'Remi Demi', 'Espresso Slatki San', 'Gerüchteküche', 'Espresso Karawane', "Karin's Radlertreff", 'SLU-Bar', 'Unders', 'Couch Potato', "Steiner's Club Lounge", 'K.St.V. Rhaetia', 'Schaumweinhäuschen', 'Schneider-Gössl Sektbar', 'Kleine Ober St. Veiter Bierstube', "Gaby's Treff", 'Biergartl', 'The Twinspub', 'Flokal', "Wistl's Cafe", "Otti's Würstel Bar", 'Hinteralm', 'Cafe Liquid', 'The Little Stage', 'Stammersdorfer Hauptprostamt